In [25]:
require 'poisson'

class Modeling
  def initialize(people_num, possibility_correct, lambda_poisson, weight)
    @people_num = people_num
    @people_half = half_num(@people_num) # 終了の値や半分の値に用いる
    @possibility_correct = possibility_correct
    @lambda_poisson = lambda_poisson # ポアソン分布の平均lambda
    @collecting_deadline = lambda_poisson * 2 # 意見収集の締め切り時刻を平均の二倍とする
    @poisson = Poisson.new(@lambda_poisson) # ポアソン分布を扱うクラスのインスタンス
    @weight = weight.to_f # 効用を計算する際の時間に対する重み。精度を重要にしたいなら重みを大きくし、速さを重要にしたいなら重みを小さくする(0.0~1.0の範囲)
  end

  # 人がいつ来るか配列で表す EXAMPLE [1, 0, 1] => 時刻0と時刻2に人がくる
  def simulate_when_people_come(people_num)
    when_people_come = Array.new(@collecting_deadline)
    people_num.times do |t|
      when_people_come[t] = 1
    end
    when_people_come.shuffle
  end


  # 一人目の意見を採用するというアルゴリズム
  def baseline_method_deciding_by_first_person_with_poisson(possibility_correct)
    average_method_utility = 0
    # 0~@collecting_deadline人来るときのみを考慮する(平均の二倍以上の人が集まる確率は限りなく0なので0に近似)
    @collecting_deadline.times do |t_people|
      # t_people人来るとき
      when_people_come = simulate_when_people_come(t_people)
      temp_probability_by_poisson = @poisson.probability{ |x| x == t_people }
      method_utility = 0 # 効用 = 精度(正解率) - 重み * かかる時間(決定する時刻 / collecting_deadline)
      when_people_come.shuffle.each_with_index do |elem, index|
        next if elem.nil? # 人が来ないときは飛ばす
        method_utility += possibility_correct - @weight * (index.to_f / @collecting_deadline)
        break # 一人目の意見しか必要ないのですぐBREAK
      end
      average_method_utility += temp_probability_by_poisson * method_utility
    end
    average_method_utility
  end

  # 引数majority_vote_people人で多数決して意見集約を行うというアルゴリズム
  def baseline_method_deciding_by_majority_vote_with_poisson(possibility_correct, majority_vote_people)
    average_method_utility = 0
    # 0~@collecting_deadline人来るときのみを考慮する(平均の二倍以上の人が集まる確率は限りなく0なので0に近似)
    @collecting_deadline.times do |t_people|
      # t_people人来るとき
      when_people_come = simulate_when_people_come(t_people)
      temp_probability_by_poisson = @poisson.probability{ |x| x == t_people }
      method_utility = 0 # 効用 = 精度(正解率) - 重み* かかる時間(決定する時刻 / collecting_deadline)
      people_count = 0 # 何人目か
      next if t_people < majority_vote_people # majority_vote_people人集まらない場合は、utility = 0
      when_people_come.shuffle.each_with_index do |elem, index|
        next if elem.nil? # 人が来ない時刻は飛ばす
        people_count += 1
        if people_count == majority_vote_people
          method_utility +=  (1 - relative_error_by_majority_vote(people_count, possibility_correct)) - @weight * (index.to_f / @collecting_deadline)
          break
        end
      end
      average_method_utility += temp_probability_by_poisson * method_utility
    end
    average_method_utility
  end

  # 先にhalf_num(引数temp_people_num)人集まった方の意見を採用するアルゴリズム
  def baseline_method_deciding_by_half_opinion_with_poisson(possibility_correct, temp_people_num)
    average_method_utility = 0
    # 0~@collecting_deadline人来るときのみを考慮する(平均の二倍以上の人が集まる確率は限りなく0なので0に近似)
    @collecting_deadline.times do |t_people|
      # t_people人来るとき
      when_people_come = simulate_when_people_come(t_people)
      temp_probability_by_poisson = @poisson.probability{ |x| x == t_people }
      method_utility = 0 # 効用 = 精度(正解率) - 重み * かかる時間(決定する時刻 / collecting_deadline)
      people_count = 0 # 何人目か
      people_count_arr = [] # EXAMPLE [4, 9, 20, 40, 45] 一人目は時刻4にきた　二人目は時刻9にきた...
      next if t_people < temp_people_num # temp_people_num人集まらない場合は、utility = 0 TODO temp_people_num人あつまらなくてもその半数は集まることはあるので厳密ではない
      when_people_come.each_with_index do |elem, index|
        next if elem.nil? # 人が来ない時刻は飛ばす
        people_count_arr << index
        people_count += 1
        if people_count == temp_people_num
          relative_error_array = relative_error_array_by_half_opinion(half_num(temp_people_num), possibility_correct)
          expected_error = relative_error_array.inject(:+)
          average_index = 0
          relative_error_array.each_with_index do |e, i| # もっと簡単にかけそう
            average_index += people_count_arr[half_num(temp_people_num) - 1 + i] * (e / expected_error)
          end
          method_utility =  (1 - expected_error) - @weight * (average_index.to_f / @collecting_deadline)
          break
        end
      end
      average_method_utility += temp_probability_by_poisson * method_utility
    end
    average_method_utility
  end

  # 誰か一人集まりかつ、時刻引数time_limitで意見収集を打ち切り,多数決で意思決定を行う
  def baseline_method_deciding_by_time_limit_with_poisson(possibility_correct, time_limit)
    average_method_utility = 0
    # 0~@collecting_deadline人来るときのみを考慮する(平均の二倍以上の人が集まる確率は限りなく0なので0に近似)
    @collecting_deadline.times do |t_people|
      # t_people人来るとき
      when_people_come = simulate_when_people_come(t_people)  # 人がいつ来るか配列で表す EXAMPLE [1, 0, 1] => 時刻0と時刻2に人がくる
      temp_probability_by_poisson = @poisson.probability{ |x| x == t_people }
      method_utility = 0 # 効用 = 精度(正解率) - かかる時間(決定する時刻 / collecting_deadline)
      people_count = 0 # 何人目か
      when_people_come.each_with_index do |elem, index|
        if index >= time_limit && people_count >= 1
          method_utility = (1 - relative_error_by_majority_vote(people_count, possibility_correct)) - @weight * (index.to_f / @collecting_deadline)
          break
        end
        next if elem.nil? # 人が来ない時刻は飛ばす
        people_count += 1
      end
      average_method_utility += temp_probability_by_poisson * method_utility
    end
    average_method_utility
  end

  def half_num(num)
    (num.to_f / 2).ceil
  end

  # nCk 実行例 5C2 = 5 * 4 / 2! = 10
  def combi(n, k)
    k = n - k if 2 * k > n
    return 1 if k == 0
    ((n - k + 1)..n).reduce(&:*) / (1..k).reduce(&:*)
  end

  # 誤差率をもとめる.先にある割合の意見(finish_num)を集めた方を意見集約の結果とするアルゴリズム
  # このアルゴリズムは必要な人数が動的に変化するので、誤差率をfinish_num個の要素にした配列で返す
  def relative_error_array_by_half_opinion(finish_num, possibility_correct) # possibility_correct => 人が正解する確率p
    relative_error = []
    finish_num.times do |t|
      relative_error << ((1 - possibility_correct)**finish_num) * (possibility_correct**t) * combi(finish_num - 1 + t, t)
    end
    relative_error # EXAMPLE (finish_num = 3 )[0.2, 0.3, 0.5] 0.2は3:0で決まるとき 0.3は3:1で決まるとき 0.5は3:2で決まるとき
  end

  # 誤差率をもとめる.people_num人で多数決を行い意見集約の結果とするアルゴリズム
  def relative_error_by_majority_vote(people_num, possibility_correct) # possibility_correct => 人が正解する確率p
    half_num = half_num(people_num)
    relative_error = 0
    half_num.times do |t|
      relative_error += (possibility_correct**t) * ((1 - possibility_correct)**(people_num - t)) * combi(people_num, t)
    end
    relative_error
  end

  # 確率pを変動させて、誤差率εの求める
  def baseline_method1(possibility_correct) # possibility_correct => 人が正解する確率p
    # 終了する人数を求める
    finish_num = @people_half
    # 誤差率を求め、返す
    relative_error_array_by_half_opinion(finish_num, possibility_correct).inject(:+)
  end

  # 確率pを変動させて、誤差率εの求める
  def baseline_method2(possibility_correct)
    # 多数決を行い誤差率を求め、返す
    relative_error_by_majority_vote(@people_num, possibility_correct)
  end

  # 誤差率εを変動させて、必要な人数Xを求める
  def baseline_method3(relative_error)
    # 必要な人数を求める 1,3,5,,,,と増やす
    people_num = 1
    # 終了する人数を求める
    temp_finish_num = half_num(people_num)
    # 必要な平均人数
    average_num = 0
    # 徐々に人数を増やしていき、誤差率を満たす結果の場合ループを終了する
    loop do
      relative_error_array = relative_error_array_by_half_opinion(temp_finish_num, @possibility_correct)
      expected_error = relative_error_array.inject(:+)
      average_num = 0
      relative_error_array.each_with_index do |e, i| # もっと簡単にかけそう
        average_num += e * (i + temp_finish_num) / expected_error
      end
      break if expected_error <= relative_error
      people_num += 2
      temp_finish_num = half_num(people_num)
    end
    # 必要な人数の平均を返す
    average_num
  end

  # 多数決を意見集約のアルゴリズムとする
  def baseline_method4(relative_error)
    # 必要な人数を求める 1,3,5,,,,と増やす
    people_num = 1
    # 徐々に人数を増やしていき、誤差率を満たす結果の場合ループを終了する
    loop do
      expected_error = relative_error_by_majority_vote(people_num, @possibility_correct)
      break if expected_error <= relative_error
      people_num += 2
    end
    # 必要な人数を返す
    people_num
  end
end


require 'rbplotly'

PEOPELE_NUM = 199
POSSIBILITY_CORRECT = 0.7
LAMBDA_POISSON = 70
WEIGHT = 0.5
DELIMITER = 100
REPEAT = 100

MAJORITY_VOTE_NUM = 5
EARLIER_NUM = 5
TIME_LIMIT = 10

# 配列の平均を算出する
def array_average(arr)
  arr.inject(:+) / arr.length
end

# 配列の分散を算出する
def array_variance(arr)
  ave = array_average(arr)
  arr.inject(0) { |a, b| a + (b - ave)**2 } / arr.length
end

model = Modeling.new(PEOPELE_NUM, POSSIBILITY_CORRECT, LAMBDA_POISSON, WEIGHT)

# 縦軸 => 効用utility, 横軸 => 人の正解する確率p
utility_possibility_x_axis = (51..99).to_a

# 引数を変えてアルゴリズムを比較

# 4種類のアルゴリズムを比較する
# 乱数を用いてるのでREPEAT回分の平均をとる
temp_utility_possibility_y_axis_by_first_person = []
temp_utility_possibility_y_axis_by_majority_vote = []
temp_utility_possibility_y_axis_by_half_opinion = []
temp_utility_possibility_y_axis_by_time_limit = []
REPEAT.times do
  temp_utility_possibility_y_axis_by_first_person << utility_possibility_x_axis.map { |e| model.baseline_method_deciding_by_first_person_with_poisson(e.to_f / DELIMITER) }
  temp_utility_possibility_y_axis_by_majority_vote << utility_possibility_x_axis.map { |e| model.baseline_method_deciding_by_majority_vote_with_poisson(e.to_f / DELIMITER, MAJORITY_VOTE_NUM) }
  temp_utility_possibility_y_axis_by_half_opinion << utility_possibility_x_axis.map { |e| model.baseline_method_deciding_by_half_opinion_with_poisson(e.to_f / DELIMITER, EARLIER_NUM) }
  temp_utility_possibility_y_axis_by_time_limit << utility_possibility_x_axis.map { |e| model.baseline_method_deciding_by_time_limit_with_poisson(e.to_f / DELIMITER, TIME_LIMIT) }
end

# 転置して平均をとる
utility_possibility_y_axis_by_first_person = temp_utility_possibility_y_axis_by_first_person.transpose.map { |e| array_variance(e) }
utility_possibility_y_axis_by_majority_vote = temp_utility_possibility_y_axis_by_majority_vote.transpose.map{ |e| array_variance(e)}
utility_possibility_y_axis_by_half_opinion = temp_utility_possibility_y_axis_by_half_opinion.transpose.map{ |e| array_variance(e)}
utility_possibility_y_axis_by_time_limit = temp_utility_possibility_y_axis_by_time_limit.transpose.map{ |e| array_variance(e)}

utility_possibility_by_first_person_trace = [{ x: utility_possibility_x_axis, y: utility_possibility_y_axis_by_first_person }]
utility_possibility_by_majority_vote_trace = [{ x: utility_possibility_x_axis, y: utility_possibility_y_axis_by_majority_vote }]
utility_possibility_by_half_opinion_trace = [{ x: utility_possibility_x_axis, y: utility_possibility_y_axis_by_half_opinion }]
utility_possibility_by_time_limit_trace = [{ x: utility_possibility_x_axis, y: utility_possibility_y_axis_by_time_limit }]

pl = Plotly::Plot.new(data: utility_possibility_by_first_person_trace + utility_possibility_by_majority_vote_trace + utility_possibility_by_half_opinion_trace + utility_possibility_by_time_limit_trace)
pl.layout.xaxis = { title: 'possibility_correct' }
pl.layout.yaxis = { title: 'utility' }
pl.layout.title = "LAMBDA_POISSON = #{LAMBDA_POISSON}, WEIGHT = #{WEIGHT}, MAJORITY_VOTE_NUM = #{MAJORITY_VOTE_NUM}, EARLIER_NUM = #{EARLIER_NUM}, TIME_LIMIT = #{TIME_LIMIT}"
pl.show


(pry):6573: warning: already initialized constant PEOPELE_NUM
(pry):6307: warning: previous definition of PEOPELE_NUM was here
(pry):6574: warning: already initialized constant POSSIBILITY_CORRECT
(pry):6308: warning: previous definition of POSSIBILITY_CORRECT was here
(pry):6575: warning: already initialized constant LAMBDA_POISSON
(pry):6309: warning: previous definition of LAMBDA_POISSON was here
(pry):6576: warning: already initialized constant WEIGHT
(pry):6310: warning: previous definition of WEIGHT was here
(pry):6577: warning: already initialized constant DELIMITER
(pry):6311: warning: previous definition of DELIMITER was here
(pry):6578: warning: already initialized constant REPEAT
(pry):6312: warning: previous definition of REPEAT was here
(pry):6580: warning: already initialized constant MAJORITY_VOTE_NUM
(pry):6314: warning: previous definition of MAJORITY_VOTE_NUM was here
(pry):6581: warning: already initialized constant EARLIER_NUM
(pry):6315: warning: previous definitio

#<Plotly::Offline::HTML:0x00007faf46288398 @id="3292d2b5-9d21-4e0c-b1c3-0cc8a885a342", @data=[{:x=>[51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99], :y=>[6.903133564390393e-07, 1.0058544510695776e-06, 8.743576351449688e-07, 8.545458810015322e-07, 9.843662882502802e-07, 7.179933950665097e-07, 6.67031852400056e-07, 6.687352690435379e-07, 9.018527896256418e-07, 8.794759502171275e-07, 8.40411035324094e-07, 9.692512283967787e-07, 8.296662428187666e-07, 1.015079040849e-06, 7.572148591732941e-07, 7.049194372086457e-07, 8.009390045885872e-07, 8.691331928070893e-07, 8.647641338931091e-07, 7.253025241186525e-07, 6.91042712124708e-07, 9.90759445520578e-07, 8.406782430509486e-07, 9.213682845487674e-07, 7.770745965671426e-07, 6.708316168699747e-07, 7.582415553082578e-07, 7.13307465630888e-07, 1.2232769343677427e-06, 7.299404404948532e-07, 7.647042360524369e-07, 7.399236615964087e-07, 7.838544318650362e-07, 8.523975909094762e-07, 9.587725174359985e-07, 8.540936714302902e-07, 8.214995579561601e-07, 9.600989881786296e-07, 8.363800943569468e-07, 8.620715928220715e-07, 8.176801862635544e-07, 5.908760029614287e-07, 8.342928853874287e-07, 1.1359467970281255e-06, 9.866353123552609e-07, 8.406564966942103e-07, 8.758864946156703e-07, 1.07208313556707e-06, 7.825793245280614e-07]}, {:x=>[51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99], :y=>[4.098694786355707e-06, 3.95268018414256e-06, 3.6952389920807673e-06, 3.887969710015939e-06, 3.866396639612122e-06, 3.3289644414862154e-06, 4.093316703732145e-06, 4.7886343330191185e-06, 3.7044183668956014e-06, 3.168465443986495e-06, 3.582148865631529e-06, 3.591645721050027e-06, 3.5604157686335047e-06, 4.522015988646508e-06, 3.903286461400712e-06, 3.53483182614533e-06, 3.6332971565058975e-06, 4.330761425930044e-06, 3.571567281642895e-06, 3.5826798431889716e-06, 3.588132349870969e-06, 3.5917753635702456e-06, 3.870844773954571e-06, 4.682970327737239e-06, 3.821209900994544e-06, 4.060334239185287e-06, 4.032706634051941e-06, 4.424109869127939e-06, 3.857810944993327e-06, 4.3486795036929384e-06, 3.5145739093890637e-06, 4.6765012563865625e-06, 3.8102048922728775e-06, 4.726413862830856e-06, 4.0147411146947954e-06, 4.335861564487646e-06, 4.488466209601579e-06, 3.87027363942759e-06, 3.5009349279446785e-06, 3.381676138475025e-06, 3.895761461213194e-06, 4.791620641209431e-06, 5.2550412885115185e-06, 4.599496899042424e-06, 3.609434250046902e-06, 4.300259349392757e-06, 3.6662009546369387e-06, 3.6806410425864725e-06, 4.018987024177144e-06]}, {:x=>[51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99], :y=>[2.9049105908587843e-06, 3.175804497580625e-06, 2.8258353797206114e-06, 2.677613460840994e-06, 3.0286509414652847e-06, 2.6728320850059986e-06, 3.5657144439821953e-06, 2.7640886990733307e-06, 2.2353322537166924e-06, 3.032162765736014e-06, 3.737450768634173e-06, 2.9840562782248893e-06, 3.186004885102203e-06, 2.7594964539086137e-06, 3.59042694557233e-06, 2.981453783731382e-06, 2.902112316385386e-06, 3.2650531267516425e-06, 2.2939334794356877e-06, 3.1312662981456633e-06, 3.2708113087740077e-06, 2.9765376113901433e-06, 3.6677425485937754e-06, 3.2979793809246294e-06, 3.147591075967712e-06, 2.7042425520936374e-06, 3.5661160428338366e-06, 3.073594571516748e-06, 3.2598894096071934e-06, 2.799599943470428e-06, 3.054842855931436e-06, 2.4973869240131083e-06, 2.9420594863854343e-06, 3.7059368801907848e-06, 3.3557759835592644e-06, 3.126141598491851e-06, 2.704683668265908e-06, 2.919079642619777e-06, 3.1060340168603806e-06, 2.925226858746295e-06, 3.5505077825732674e-06, 2.6518809118798193e-06, 2.8186769474116675e-06, 4

#<CZTop::Socket::PUB:0x7faf47111b00 last_endpoint="tcp://127.0.0.1:55147">